In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from sklearn.model_selection import train_test_split
from shutil import copyfile

mainDataDir='/kaggle/input/multi-cancer/Multi Cancer/Lymphoma'
lymphCLLDir=os.path.join(mainDataDir,'lymph_cll')
lymphFLDir=os.path.join(mainDataDir,'lymph_fl')
lymphMCLDir=os.path.join(mainDataDir,'lymph_mcl')

In [3]:
trainDir = '/kaggle/working/training_data'
valDir = '/kaggle/working/validation_data'

In [4]:
os.makedirs(trainDir,exist_ok=True)
os.makedirs(valDir,exist_ok=True)

In [5]:
def split_and_copy(class_dir, train_output_dir, val_output_dir):
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith(('jpg', 'jpeg', 'png'))]

    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(train_output_dir, exist_ok=True)
    os.makedirs(val_output_dir, exist_ok=True)

    for img in train_images:
        copyfile(img, os.path.join(train_output_dir, os.path.basename(img)))

    for img in val_images:
        copyfile(img, os.path.join(val_output_dir, os.path.basename(img)))

split_and_copy(lymphCLLDir, os.path.join(trainDir, 'lymph_cll'), os.path.join(valDir, 'lymph_cll'))
split_and_copy(lymphFLDir, os.path.join(trainDir, 'lymph_fl'), os.path.join(valDir, 'lymph_fl'))
split_and_copy(lymphMCLDir, os.path.join(trainDir, 'lymph_mcl'), os.path.join(valDir, 'lymph_mcl'))

## Data Augmentation

In [53]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    '/kaggle/working/training_data',
    target_size=(224,224),
    class_mode='sparse',
    batch_size=64
)

val_generator=val_datagen.flow_from_directory(
    '/kaggle/working/validation_data',
    target_size=(224,224),
    class_mode='sparse',
    batch_size=64
)

Found 12000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.


## Training the Data

In [54]:
val_generator.class_indices

{'lymph_cll': 0, 'lymph_fl': 1, 'lymph_mcl': 2}

In [55]:
val_generator.class_mode

'sparse'

In [56]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Input,InputLayer,GlobalAveragePooling1D,BatchNormalization

In [57]:
numClasses=len(train_generator.class_indices)

In [58]:
imageShape=(224,224,3)

In [80]:
from tensorflow.keras.regularizers import l2

model=Sequential([
    Input(shape=imageShape),
    Conv2D(64,(3,3),activation='relu',kernel_regularizer=l2(0.001)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.2),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(numClasses,activation='softmax')
])

In [81]:
from keras.optimizers import Adam,SGD

In [82]:
optimizer = Adam(learning_rate=0.001)  # Reduce learning rate
model.compile(
    optimizer=optimizer,
#     optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [83]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 111, 111, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 109, 109, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,798,915 (10.68 MB)

 Trainable params: 2,798,915 (10.68 MB)

 Non-trainable params: 0 (0.00 B)

In [84]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
es=EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True,mode="min")
rlronp=ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=1,verbose=1)
callbacks=[es, rlronp]

In [85]:
batch_size=16
steps_per_epoch = len(train_generator) // batch_size  
validation_steps = len(val_generator) // batch_size  
history = model.fit(train_generator,validation_data=val_generator,epochs=100,steps_per_epoch=steps_per_epoch,validation_steps=validation_steps,callbacks=callbacks)

Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.3370 - loss: 1.2512 - val_accuracy: 0.4141 - val_loss: 1.1177 - learning_rate: 0.0010
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - accuracy: 0.4038 - loss: 1.1052 - val_accuracy: 0.3750 - val_loss: 1.1108 - learning_rate: 0.0010
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - accuracy: 0.3575 - loss: 1.1131 - val_accuracy: 0.4453 - val_loss: 1.0760 - learning_rate: 0.0010
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - accuracy: 0.4217 - loss: 1.0797 - val_accuracy: 0.5703 - val_loss: 1.0287 - learning_rate: 0.0010
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4653 - loss: 1.0427
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
11/11 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - accuracy: 0.4616 - loss: 1.0437 - val_accuracy: 0.3672 - val_loss: 1.2090 - learning_rate: 0.0010
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4250 - loss: 1.0768
Epoch 6: Redu

In [86]:
# Evaluate on the training generator
train_results = model.evaluate(train_generator,batch_size=64)
print("Train loss, Train accuracy:", train_results)

# Evaluate on the validation generator
val_results = model.evaluate(val_generator)
print("Validation loss, Validation accuracy:", val_results)

188/188 ━━━━━━━━━━━━━━━━━━━━ 292s 2s/step - accuracy: 0.5111 - loss: 0.9978
Train loss, Train accuracy: [0.9936739206314087, 0.5150833129882812]
47/47 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.5016 - loss: 1.0126
Validation loss, Validation accuracy: [1.010684847831726, 0.5120000243186951]


In [87]:
def savingModel(model, model_name="model"): 
    model_filename = f"{model_name}.h5"
    model.save(model_filename)
    print(f"Model saved as {model_filename}")

In [88]:
savingModel(model,model_name="LymphCancer")

Model saved as LymphCancer.h5


In [89]:
from keras.models import load_model

In [90]:
def loadingModel(model_name="model"): 
    try:
        model = load_model(f"{model_name}.h5")
        model.summary()  # Print model summary if needed
        return model  # Return the loaded model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [92]:
loadedModel=loadingModel("LymphCancer")

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 111, 111, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 109, 109, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,798,917 (10.68 MB)

 Trainable params: 2,798,915 (10.68 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [93]:
def loadImg(imgPath): 
    img = tf.keras.preprocessing.image.load_img(imgPath, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array=img_array/255.0
    return img_array

In [94]:
import tensorflow as tf

In [95]:
def predictImagesInFolder(testFolder, class_name, model_name): 
    savedModel = loadingModel(model_name)
    print(savedModel)

    if savedModel is None:
        print("Model loading failed.")
        return

    for subdir, dirs, files in os.walk(testFolder): 
        if subdir == testFolder: 
            continue
        trueLabel = os.path.basename(subdir)
        for file in files: 
            img_path = os.path.join(subdir, file)
            img = loadImg(img_path)
            
            # Debugging prediction probabilities
            predictions = savedModel.predict(img)
#             print(f"Prediction Probabilities for {file}: {predictions}")

            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class = class_name[predicted_class_index[0]]
            print(f"Predicted class: {predicted_class} File: {file}")

In [96]:
def testFolder(folder): 
    testFolder=f"{folder}"
    return testFolder

In [97]:
LymphCancerTestFolder=testFolder("/kaggle/working/validation_data")

In [98]:
train_generator.class_indices

{'lymph_cll': 0, 'lymph_fl': 1, 'lymph_mcl': 2}

In [99]:
LymphCancerClassname={0:'lymph_cll', 1:'lymph_fl', 2:'lymph_mcl'}

In [ ]:
predictImagesInFolder(LymphCancerTestFolder,LymphCancerClassname,model_name="LymphCancer")

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 111, 111, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 109, 109, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,798,917 (10.68 MB)

 Trainable params: 2,798,915 (10.68 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

<Sequential name=sequential_7, built=True>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Predicted class: lymph_fl File: lymph_fl_4329.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted class: lymph_fl File: lymph_fl_3710.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted class: lymph_fl File: lymph_fl_0439.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted class: lymph_fl File: lymph_fl_1963.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted class: lymph_fl File: lymph_fl_1442.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted class: lymph_fl File: lymph_fl_4592.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted class: lymph_fl File: lymph_fl_0246.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted class: lymph_fl File: lymph_fl_4057.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted class: lymph_fl File: lymph_fl_1177.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted class: lymph_fl File: lymph_fl_4402.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted class: lymph_cll File: lymph